$$
\newcommand{\fudm}[2]{\frac{\mathrm{D} #1}{\mathrm{D} #2}}
\newcommand{\pad}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\ppad}[2]{\frac{\partial^2 #1}{\partial #2^2}}
\newcommand{\ppadd}[3]{\frac{\partial^2 #1}{\partial #2 \partial #3}}
\newcommand{\nnabla}{\nabla^2}
\newcommand{\eps}{\epsilon}
\newcommand{\vdetail}[1]{\vb{#1}=\begin{pmatrix}#1_1\\#1_2\\#1_3\end{pmatrix}}
\newcommand{\vb}[1]{\mathbf{#1}}
\newcommand{\va}[1]{\vec{#1}}
\newcommand{\tb}[1]{\underline{\underline{\mathbf{#1}}}}
\newcommand{\fud}[2]{\frac{\mathrm{d} #1}{\mathrm{d} #2}}
$$

# 4. Spherical Waves

In spherical coordinates $(r,\phi,\theta)$ with 

\begin{eqnarray}
x&=&r \sin \theta \cos \phi\tag{4.1a}\\
y&=&r \sin \theta \sin \phi\tag{4.1b}\\
z&=&r \cos \theta \tag{4.1c}
\end{eqnarray}

the Laplacian operator is

$$\nabla^2=\ppad{}{r} + \frac{2}{r}\pad{}{r} +\frac{1}{r^2 \sin \theta}\pad{}{\theta}\left(\sin \theta \pad{}{\theta}\right) +
\frac{1}{r^2 \sin^2 \theta} \ppad{}{\phi} \tag{4.2}
$$

If we assume spherical symmetry the the acoustic pressure is a function of $r$ only and the Laplacian simplifies to

$$\nabla^2=\ppad{}{r}+\frac{2}{r}\pad{}{r}\tag{4.3}\quad .$$

Thus the wave equation becomes

$$\ppad{p}{r}+\frac{2}{r}\pad{p}{r}=\frac{1}{c^2}\ppad{p}{t}\tag{4.4}\quad .$$

We can rewrite Eq. (4.4) as

$$\ppad{(p r)}{r}=\frac{1}{c^2}\ppad{(r p)}{t}\tag{4.5}\quad ,$$

which resembles a plane wave equation for the product of the variable $p$ and $r$. Thus the solution to this equation is

$$p r = f_1(c t - r) + f_2 (c t +r)$$

or 

$$p = \frac{1}{r} f_1(c t - r) + \frac{1}{r} f_2 (c t +r)\tag{4.6}\quad ,$$

for all $r>0$, the solution is not valid at $r=0$. Equation (4.6) is an outgoing and incoming wave where the amplitude is proportional to $1/r$. The reason for failure of Eq. (4.6) of the outgoing wave is that sound source is not accounted for. In contrast the incoming wave diverges at $r=0$. The reason why our equation becomes invalid is that the small amplitude limit is not valid anymore, nonlinearities build up and we need a better description which accounts for *finite amplitude effects*.


